In [1]:
!pip install spektral

     |████████████████████████████████| 122 kB 390 kB/s 
     |████████████████████████████████| 454.3 MB 8.8 kB/s 
     |████████████████████████████████| 4.0 MB 17.8 MB/s 
     |████████████████████████████████| 1.2 MB 31.5 MB/s 
     |████████████████████████████████| 6.0 MB 26.5 MB/s 
     |████████████████████████████████| 4.0 MB 31.5 MB/s 
     |████████████████████████████████| 462 kB 31.3 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Attempting uninstall: h5py
    Found existing installatio

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import Adam

from spektral.data import DisjointLoader
from spektral.datasets import TUDataset
from spektral.models import GeneralGNN

In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
################################################################################
# Config
################################################################################
batch_size = 32
learning_rate = 0.01
epochs = 40

In [5]:
################################################################################
# Load data
################################################################################
data = TUDataset("PROTEINS")

# Train/test split
np.random.shuffle(data)
split = int(0.8 * len(data))
data_tr, data_te = data[:split], data[split:]

# Data loaders
loader_tr = DisjointLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_te = DisjointLoader(data_te, batch_size=batch_size)

100%|█████████████████████████████████████████| 447k/447k [00:01<00:00, 331kB/s]


Successfully loaded PROTEINS.


In [6]:
data[0]

Graph(n_nodes=62, n_node_features=4, n_edge_features=None, n_labels=2)

In [7]:
################################################################################
# Build model
################################################################################
model = GeneralGNN(data.n_labels, activation="softmax")
optimizer = Adam(learning_rate)
loss_fn = CategoricalCrossentropy()

In [8]:
################################################################################
# Fit model
################################################################################
@tf.function(input_signature=loader_tr.tf_signature(), experimental_relax_shapes=True)
def train_step(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    acc = tf.reduce_mean(categorical_accuracy(target, predictions))
    return loss, acc


def evaluate(loader):
    output = []
    step = 0
    while step < loader.steps_per_epoch:
        step += 1
        inputs, target = loader.__next__()
        pred = model(inputs, training=False)
        outs = (
            loss_fn(target, pred),
            tf.reduce_mean(categorical_accuracy(target, pred)),
            len(target),  # Keep track of batch size
        )
        output.append(outs)
        if step == loader.steps_per_epoch:
            output = np.array(output)
            return np.average(output[:, :-1], 0, weights=output[:, -1])


epoch = step = 0
results = []
for batch in loader_tr:
    step += 1
    loss, acc = train_step(*batch)
    results.append((loss, acc))
    if step == loader_tr.steps_per_epoch:
        step = 0
        epoch += 1
        results_te = evaluate(loader_te)
        print(
            "Ep. {} - Loss: {:.3f} - Acc: {:.3f} - Test loss: {:.3f} - Test acc: {:.3f}".format(
                epoch, *np.mean(results, 0), *results_te
            )
        )
        results = []

Ep. 1 - Loss: 0.663 - Acc: 0.603 - Test loss: 4.251 - Test acc: 0.439
Ep. 2 - Loss: 0.546 - Acc: 0.736 - Test loss: 4.416 - Test acc: 0.408
Ep. 3 - Loss: 0.521 - Acc: 0.746 - Test loss: 1.912 - Test acc: 0.408
Ep. 4 - Loss: 0.528 - Acc: 0.737 - Test loss: 0.728 - Test acc: 0.583
Ep. 5 - Loss: 0.521 - Acc: 0.739 - Test loss: 1.281 - Test acc: 0.448
Ep. 6 - Loss: 0.514 - Acc: 0.745 - Test loss: 0.660 - Test acc: 0.623
Ep. 7 - Loss: 0.511 - Acc: 0.753 - Test loss: 0.604 - Test acc: 0.722
Ep. 8 - Loss: 0.504 - Acc: 0.759 - Test loss: 0.508 - Test acc: 0.798
Ep. 9 - Loss: 0.501 - Acc: 0.761 - Test loss: 0.592 - Test acc: 0.731
Ep. 10 - Loss: 0.502 - Acc: 0.763 - Test loss: 0.662 - Test acc: 0.673
Ep. 11 - Loss: 0.510 - Acc: 0.749 - Test loss: 0.528 - Test acc: 0.744
Ep. 12 - Loss: 0.486 - Acc: 0.772 - Test loss: 0.898 - Test acc: 0.596
Ep. 13 - Loss: 0.502 - Acc: 0.775 - Test loss: 0.587 - Test acc: 0.749
Ep. 14 - Loss: 0.497 - Acc: 0.766 - Test loss: 1.032 - Test acc: 0.516
Ep. 15 - Loss: 

In [9]:
################################################################################
# Evaluate model
################################################################################
results_te = evaluate(loader_te)
print("Final results - Loss: {:.3f} - Acc: {:.3f}".format(*results_te))

Final results - Loss: 0.583 - Acc: 0.713
